<a href="https://colab.research.google.com/github/P3109/Public/blob/main/Value%20Tables/make-value-tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Value tables of 8-bit floating point formats

This code allows the reader to experiment with properties of FP8 formats, and is produced in conjunction with the work of the IEEE P3109 working group on
floating point formats for machine learning, although is *not* an official output of that group.

The public outputs of of the group are available at https://github.com/P3109/Public and the interim report is at [PDF](https://github.com/P3109/Public/blob/main/Shared%20Reports/P3109%20WG%20Interim%20report.pdf)


In [1]:
%pip install pandas jinja2 airium ipytest gfloat==0.2.1

Note: you may need to restart the kernel to use updated packages.


## FormatInfo

A small dataclass holding format information.  
See https://gfloat.readthedocs.io/en/latest/#gfloat.FormatInfo


In [2]:
import pandas
import numpy as np
import gfloat
from gfloat import FloatClass, FormatInfo, FloatValue, decode_float 
from gfloat.formats import *

def df_left_align(df):
  """
  Left-align dataframe for display
  """
  return df.style.set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])])

other_formats = [format_info_ocp_e5m2, format_info_ocp_e4m3]
p3109_formats = [format_info_p3109(i) for i in range(1,8)]
p3109_and_ocp_formats = p3109_formats + other_formats
sub32_formats = p3109_and_ocp_formats + [format_info_binary16, format_info_bfloat16, format_info_binary32]

# hacky fix to names - can be removed after https://github.com/graphcore-research/gfloat/pull/22 is released
for fi in sub32_formats:
  fi.name = fi.name.replace('format_info_','')

df = pandas.DataFrame(sub32_formats).set_index('name')


print('name: Short name for the format, e.g. binary32, bfloat16')
print('k: Number of bits in the format')
print('precision: Number of significand bits (including implicit leading bit encoded by exponent>0)')
print('emax: Largest exponent, emax, which shall equal floor(log_2(maxFinite))')
print('has_nz: Has negative zero (following e.g. LLVM Float8E5M2FNUZ)')
print('has_infs: Set if format includes +/- Infinity')
print('num_high_nans: Number of NaNs that are encoded in the highest encodings for each sign')
print('has_subnormals: Set if format encodes subnormals')
print('is_signed: Set if the format has a sign bit')
print('is_twos_complement: Set if the format uses two''s complement encoding for the significand')
display(df_left_align(df))


name: Short name for the format, e.g. binary32, bfloat16
k: Number of bits in the format
precision: Number of significand bits (including implicit leading bit encoded by exponent>0)
emax: Largest exponent, emax, which shall equal floor(log_2(maxFinite))
has_nz: Has negative zero (following e.g. LLVM Float8E5M2FNUZ)
has_infs: Set if format includes +/- Infinity
num_high_nans: Number of NaNs that are encoded in the highest encodings for each sign
has_subnormals: Set if format encodes subnormals
is_signed: Set if the format has a sign bit
is_twos_complement: Set if the format uses twos complement encoding for the significand


,k,precision,emax,has_nz,has_infs,num_high_nans,has_subnormals,is_signed,is_twos_complement
name,,,,,,,,,
p3109_p1,8,1,63,False,True,0,True,True,False
p3109_p2,8,2,31,False,True,0,True,True,False
p3109_p3,8,3,15,False,True,0,True,True,False
p3109_p4,8,4,7,False,True,0,True,True,False
p3109_p5,8,5,3,False,True,0,True,True,False
p3109_p6,8,6,1,False,True,0,True,True,False
p3109_p7,8,7,0,False,True,0,True,True,False
ocp_e5m2,8,3,15,True,True,3,True,True,False
ocp_e4m3,8,4,8,True,False,1,True,True,False


## Enumerate all values for a given format

In [3]:
# Get the FormatInfo
fi = format_info_p3109(precision=3)

# Generate values as a list of FloatValues
values = [decode_float(fi, i) for i in range(256)]

# Convert to dataframe (and hide the FormatInfo)
df_left_align(pandas.DataFrame(values)).hide().hide(subset=[*range(9,120), *range(131,251)])


print('ival:Integer code point')
print('fval:Value (as python float)')
print('exp:Raw exponent without bias')
print('expval:Exponent, bias subtracted')
print('significand:Significand as an integer')
print('fsignificand:Significand as a float in the range [0,2)')
print('signbit:1 => negative, 0 => positive')
print('fclass: ZERO, SUBNORMAL, NORMAL, INFINITE, NAN')


with pandas.option_context("render.max_rows", 9):
  df = pandas.DataFrame(values)
  display(df_left_align(df[0:8]).hide())
  display(df_left_align(df[124:134]).hide())
  display(df_left_align(df[249:]).hide())


ival:Integer code point
fval:Value (as python float)
exp:Raw exponent without bias
expval:Exponent, bias subtracted
significand:Significand as an integer
fsignificand:Significand as a float in the range [0,2)
signbit:1 => negative, 0 => positive
fclass: ZERO, SUBNORMAL, NORMAL, INFINITE, NAN


ival,fval,exp,expval,significand,fsignificand,signbit,fclass
0,0.000000,0,-15,0,0.000000,0,FloatClass.ZERO
1,0.000008,0,-15,1,0.250000,0,FloatClass.SUBNORMAL
2,0.000015,0,-15,2,0.500000,0,FloatClass.SUBNORMAL
3,0.000023,0,-15,3,0.750000,0,FloatClass.SUBNORMAL
4,0.000031,1,-15,0,1.000000,0,FloatClass.NORMAL
5,0.000038,1,-15,1,1.250000,0,FloatClass.NORMAL
6,0.000046,1,-15,2,1.500000,0,FloatClass.NORMAL
7,0.000053,1,-15,3,1.750000,0,FloatClass.NORMAL


ival,fval,exp,expval,significand,fsignificand,signbit,fclass
124,32768.000000,31,15,0,1.000000,0,FloatClass.NORMAL
125,40960.000000,31,15,1,1.250000,0,FloatClass.NORMAL
126,49152.000000,31,15,2,1.500000,0,FloatClass.NORMAL
127,inf,31,15,3,1.750000,0,FloatClass.INFINITE
128,nan,0,-15,0,0.000000,1,FloatClass.NAN
129,-0.000008,0,-15,1,0.250000,1,FloatClass.SUBNORMAL
130,-0.000015,0,-15,2,0.500000,1,FloatClass.SUBNORMAL
131,-0.000023,0,-15,3,0.750000,1,FloatClass.SUBNORMAL
132,-0.000031,1,-15,0,1.000000,1,FloatClass.NORMAL
...,...,...,...,...,...,...,...


ival,fval,exp,expval,significand,fsignificand,signbit,fclass
249,-20480.000000,30,14,1,1.250000,1,FloatClass.NORMAL
250,-24576.000000,30,14,2,1.500000,1,FloatClass.NORMAL
251,-28672.000000,30,14,3,1.750000,1,FloatClass.NORMAL
252,-32768.000000,31,15,0,1.000000,1,FloatClass.NORMAL
253,-40960.000000,31,15,1,1.250000,1,FloatClass.NORMAL
254,-49152.000000,31,15,2,1.500000,1,FloatClass.NORMAL
255,-inf,31,15,3,1.750000,1,FloatClass.INFINITE


## Collect value statistics for formats

Stats collected are:
 - lt1: Number of values with `0 < x < 1`
 - gt1: Number of values with `1<=x<Inf`
 - rt16: Round-trips to IEEE binary16
 - rt32: Round-trips to IEEE binary32 (True for all current formats, would not be true if emax(8) were 2**(-1/2).
 - maxNormal: Largest finite normal value, NaN if all finite values are subnormal
 - minSubnormal: Smallest positive subnormal value, NaN if no finite values are subnormal


In [4]:

def collect_stats(fi : FormatInfo):
  # Generate all values
  values = [decode_float(fi, i) for i in range(256)]
  df = pandas.DataFrame(values)
  
  # Extract format information parameters
  E=fi.expBits
  S=fi.tSignificandBits

  # Compute statistics: lt1,ge1
  fval = df['fval']
  total_01 = fval.between(0, 1, inclusive='neither').sum()
  total_1Inf = fval.between(1, np.inf, inclusive='left').sum()

  # Compute statistics: maxFinite,minFinite
  finite_vals = fval[np.isfinite(fval)]
  maxFinite = finite_vals.loc[finite_vals.idxmax()]
  minFinite = finite_vals.loc[finite_vals.idxmin()]

  # Compute statistics: maxNormal,minNormal
  normal_vals = fval[(df['fclass'] == FloatClass.NORMAL) & (fval>0)]
  maxNormal = normal_vals.loc[normal_vals.idxmax()] if normal_vals.any() else np.nan
  minNormal = normal_vals.loc[normal_vals.idxmin()] if normal_vals.any() else np.nan

  # Compute statistics: minSubnormal
  pos_subnormal = fval[(df['fclass'] == FloatClass.SUBNORMAL) & (fval>0)]
  maxSubnormal = pos_subnormal.loc[pos_subnormal.idxmax()] if pos_subnormal.any() else np.nan
  minSubnormal = pos_subnormal.loc[pos_subnormal.idxmin()] if pos_subnormal.any() else np.nan

  # Compute roundtrips: rt16, rt32
  with np.errstate(over='ignore'):
    rt16 = (np.float64(np.float16(fval)) == np.float64(fval)) | ~np.isfinite(fval)
    rt32 = (np.float64(np.float32(fval)) == np.float64(fval)) | ~np.isfinite(fval)

  rt16 = rt16.all()
  rt32 = rt32.all()
  assert rt32 # If not, we should include rt32 in the table
  
  # Assemble tuple
  return dict(name=fi.name,P=fi.precision,E=E,T=S,
            lt1=total_01,ge1=total_1Inf,
            rt16=rt16,
            maxFinite=maxFinite,minFinite=minFinite,
            maxNormal=maxNormal,minNormal=minNormal,
            minSubnormal=minSubnormal,maxSubnormal=maxSubnormal)

print('P: Precision')
print('E: Exponent field width in bits')
print('T: Trailing significand field width in bits')
print('lt1: Number of values <code>x</code> such that <code>0 < x < 1</code>')
print('ge1: Number of values <code>x</code> such that <code>1 <= x < Inf</code>')
print('rt16: Round-trips to IEEE binary16')
print('maxFinite: Largest finite value')
print('minFinite: Smallest finite value')
print('maxNormal: Largest finite normal value, NaN if all finite values are subnormal')
print('minNormal: Smallest positive normal value, NaN if all finite values are subnormal')
print('minSubnormal: Smallest positive subnormal value, NaN if no finite values are subnormal')
print('maxSubnormal: Largest subnormal value, NaN if no finite values are subnormal')

stats = [collect_stats(fi) for fi in p3109_and_ocp_formats]
df = pandas.DataFrame(stats)
df_left_align(df).hide()

P: Precision
E: Exponent field width in bits
T: Trailing significand field width in bits
lt1: Number of values <code>x</code> such that <code>0 < x < 1</code>
ge1: Number of values <code>x</code> such that <code>1 <= x < Inf</code>
rt16: Round-trips to IEEE binary16
maxFinite: Largest finite value
minFinite: Smallest finite value
maxNormal: Largest finite normal value, NaN if all finite values are subnormal
minNormal: Smallest positive normal value, NaN if all finite values are subnormal
minSubnormal: Smallest positive subnormal value, NaN if no finite values are subnormal
maxSubnormal: Largest subnormal value, NaN if no finite values are subnormal


name,P,E,T,lt1,ge1,rt16,maxFinite,minFinite,maxNormal,minNormal,minSubnormal,maxSubnormal
p3109_p1,1,7,0,62,64,False,9223372036854775808.000000,-9223372036854775808.000000,9223372036854775808.000000,0.000000,nan,nan
p3109_p2,2,6,1,63,63,False,2147483648.000000,-2147483648.000000,2147483648.000000,0.000000,0.000000,0.000000
p3109_p3,3,5,2,63,63,True,49152.000000,-49152.000000,49152.000000,0.000031,0.000008,0.000023
p3109_p4,4,4,3,63,63,True,224.000000,-224.000000,224.000000,0.007812,0.000977,0.006836
p3109_p5,5,3,4,63,63,True,15.000000,-15.000000,15.000000,0.125000,0.007812,0.117188
p3109_p6,6,2,5,63,63,True,3.875000,-3.875000,3.875000,0.500000,0.015625,0.484375
p3109_p7,7,1,6,63,63,True,1.968750,-1.968750,1.968750,1.000000,0.015625,0.984375
ocp_e5m2,3,5,2,59,64,True,57344.000000,-57344.000000,57344.000000,0.000061,0.000015,0.000046
ocp_e4m3,4,4,3,55,71,True,448.000000,-448.000000,448.000000,0.015625,0.001953,0.013672


## Value statistics as exact fractions * 2^e


In [5]:
df_pow2 = df.copy()
for field in ("maxFinite","minFinite","maxNormal","minNormal","minSubnormal","maxSubnormal"):
  df_pow2[field] = df_pow2[field].map(gfloat.printing.float_pow2str)
df_left_align(df_pow2).hide()

name,P,E,T,lt1,ge1,rt16,maxFinite,minFinite,maxNormal,minNormal,minSubnormal,maxSubnormal
p3109_p1,1,7,0,62,64,False,1*2^63,-1*2^63,1*2^63,1*2^-62,nan,nan
p3109_p2,2,6,1,63,63,False,1*2^31,-1*2^31,1*2^31,1*2^-31,1*2^-32,1*2^-32
p3109_p3,3,5,2,63,63,True,3/2*2^15,-3/2*2^15,3/2*2^15,1*2^-15,1*2^-17,3/2*2^-16
p3109_p4,4,4,3,63,63,True,7/4*2^7,-7/4*2^7,7/4*2^7,1*2^-7,1*2^-10,7/4*2^-8
p3109_p5,5,3,4,63,63,True,15/8*2^3,-15/8*2^3,15/8*2^3,1*2^-3,1*2^-7,15/8*2^-4
p3109_p6,6,2,5,63,63,True,31/16*2^1,-31/16*2^1,31/16*2^1,1*2^-1,1*2^-6,31/16*2^-2
p3109_p7,7,1,6,63,63,True,63/32*2^0,-63/32*2^0,63/32*2^0,1*2^0,1*2^-6,63/32*2^-1
ocp_e5m2,3,5,2,59,64,True,7/4*2^15,-7/4*2^15,7/4*2^15,1*2^-14,1*2^-16,3/2*2^-15
ocp_e4m3,4,4,3,55,71,True,7/4*2^8,-7/4*2^8,7/4*2^8,1*2^-6,1*2^-9,7/4*2^-7


## Value Tables

Print one-line summaries of each float value in the form
```text
Code Binary     = Exact binary E  = Float Value
0x21 0_0100_001 = +0b1.001*2^-4   = ~0.0703
```

In [6]:
from gfloat import float_tilde_unless_roundtrip_str

def str_bits_with_underscores(fi, fv):
    # 0_1011110_
    if fi.tSignificandBits == 0:
        return f"{fv.signbit}_{fv.exp:0{fi.expBits}b}_"

    # 0__1011110
    if fi.expBits == 0:
        return f"{fv.signbit}__{fv.significand:0{fi.tSignificandBits}b}"

    # 0_101_1110
    return (
        f"{fv.signbit}_{fv.exp:0{fi.expBits}b}_{fv.significand:0{fi.tSignificandBits}b}"
    )

def str_tablerow(fi : FormatInfo, fv : FloatValue, vs_width=14, vs_d=8):
  """
  Create a string of the form
    0x41 0_10000_01 = +0b1.01*2^0   = 1.25
  """
  text = []

  # 0x45 0_1000_101
  text.append(f'0x{fv.ival:02x} {str_bits_with_underscores(fi,fv)}')
  
  finite_nonzero = np.isfinite(fv.fval) and fv.fval != 0
  
  #  = +0b1.101*2^-7 =
  if finite_nonzero:
    b = '0' if fv.fclass == FloatClass.SUBNORMAL else '1'
    signstr = "-" if fv.signbit else "+"
    binary_pow2 = f'{signstr}0b{b}.{fv.significand:0{fi.tSignificandBits}b}*2^{fv.expval:<3}'
    text.append(binary_pow2)

  # 1.125
  text.append(float_tilde_unless_roundtrip_str(fv.fval, width=vs_width, d=vs_d))

  # Return tuple
  return " = ".join(text)

for fi in (format_info_ocp_e5m2, format_info_p3109(1)):
  print()
  print(fi.name)
  for i in (0x00, 0x01, 0x07, 0x21, 0x40, 0x41, 0x7e, 0x7f, 0x80, 0x81, 0xe6, 0xfe,0xff):
    print(str_tablerow(fi, decode_float(fi, i), vs_width=8, vs_d=4))



ocp_e5m2
0x00 0_00000_00 = 0.0
0x01 0_00000_01 = +0b0.01*2^-14 = ~1.526e-05
0x07 0_00001_11 = +0b1.11*2^-14 = ~0.0001
0x21 0_01000_01 = +0b1.01*2^-7  = ~0.0098
0x40 0_10000_00 = +0b1.00*2^1   = 2.0
0x41 0_10000_01 = +0b1.01*2^1   = 2.5
0x7e 0_11111_10 = nan
0x7f 0_11111_11 = nan
0x80 1_00000_00 = -0.0
0x81 1_00000_01 = -0b0.01*2^-14 = ~-1.526e-05
0xe6 1_11001_10 = -0b1.10*2^10  = -1536.0
0xfe 1_11111_10 = nan
0xff 1_11111_11 = nan

p3109_p1
0x00 0_0000000_ = 0.0
0x01 0_0000001_ = +0b1.0*2^-62 = ~2.168e-19
0x07 0_0000111_ = +0b1.0*2^-56 = ~1.388e-17
0x21 0_0100001_ = +0b1.0*2^-30 = ~9.313e-10
0x40 0_1000000_ = +0b1.0*2^1   = 2.0
0x41 0_1000001_ = +0b1.0*2^2   = 4.0
0x7e 0_1111110_ = +0b1.0*2^63  = ~9.223e+18
0x7f 0_1111111_ = inf
0x80 1_0000000_ = nan
0x81 1_0000001_ = -0b1.0*2^-62 = ~-2.168e-19
0xe6 1_1100110_ = -0b1.0*2^39  = ~-5.498e+11
0xfe 1_1111110_ = -0b1.0*2^63  = ~-9.223e+18
0xff 1_1111111_ = -inf


## Generate HTML Value Tables

In [7]:
import airium
from IPython.display import HTML

def table_style(fv):
    """
    Select from the table entry styles defined in CSS below.
    """
    if fv.fclass == FloatClass.SUBNORMAL:
        return "subnormal"

    if fv.fclass == FloatClass.NORMAL:
        return "normal"

    if fv.fclass == FloatClass.ZERO and not fv.signbit:
        return "normal"

    # Everyting else is special
    return "special"

def mktbl(fi : FormatInfo, short=False, **kw):
  # Make tables
  cols = 4
  rows = 256//cols

  style = f'''
  body {{
  }}
  table {{
    width:100%;
    margin: 0pt;
    font-family: monospace;
    font-size: small;
    font-weight: bold;
    border-collapse: collapse;
  }}

  tr.blankrow {{
    height: 4ex;
    vertical-align: top;
  }}
  
  td {{
    text-align: left;
    border: solid 2px #ccc;
    width: {98/cols}%;
  }}
  
  .special {{
    color: #874723;
  }}
  
  .subnormal {{
    color: #0121a7;
  }}
  
  .normal {{
  }}
  
  pre {{
    margin: 1pt 1pt 1pt 13pt;
    display: inline;
  }}
'''

  title = f"FP8 Value Table, {fi.name}"
  a = airium.Airium()
  a('<!DOCTYPE html>')
  with a.html():
    with a.head():
        # a.meta('http-equiv="refresh" content="1"') # for rapid testing
        a.meta(charset="utf-8")
        a.title(_t=title)
        a.style(_t=style)

    with a.body():
        a.h3(_t=title)

        with a.table():
          for i in range(0,rows):
            if short and (16 <= i < 60):
              if i == 16:
                a.tr(klass='blankrow').td('...')
              continue
            trklass='blankrow' if i > 0 and i%16 == 0 else ''
            with a.tr(klass=trklass):
              for n in range(i,256,rows):
                fv = decode_float(fi, n)
                text = str_tablerow(fi, fv, **kw)
                a.td(klass=table_style(fv)).pre(_t=text)

  return str(a)

HTML(mktbl(format_info_ocp_e5m2, short=True, vs_width=8, vs_d=3))

0x00 0_00000_00 = 0.0,0x40 0_10000_00 = +0b1.00*2^1 = 2.0,0x80 1_00000_00 = -0.0,0xc0 1_10000_00 = -0b1.00*2^1 = -2.0
0x01 0_00000_01 = +0b0.01*2^-14 = ~1.53e-05,0x41 0_10000_01 = +0b1.01*2^1 = 2.5,0x81 1_00000_01 = -0b0.01*2^-14 = ~-1.53e-05,0xc1 1_10000_01 = -0b1.01*2^1 = -2.5
0x02 0_00000_10 = +0b0.10*2^-14 = ~3.05e-05,0x42 0_10000_10 = +0b1.10*2^1 = 3.0,0x82 1_00000_10 = -0b0.10*2^-14 = ~-3.05e-05,0xc2 1_10000_10 = -0b1.10*2^1 = -3.0
0x03 0_00000_11 = +0b0.11*2^-14 = ~4.58e-05,0x43 0_10000_11 = +0b1.11*2^1 = 3.5,0x83 1_00000_11 = -0b0.11*2^-14 = ~-4.58e-05,0xc3 1_10000_11 = -0b1.11*2^1 = -3.5
0x04 0_00001_00 = +0b1.00*2^-14 = ~6.1e-05,0x44 0_10001_00 = +0b1.00*2^2 = 4.0,0x84 1_00001_00 = -0b1.00*2^-14 = ~-6.1e-05,0xc4 1_10001_00 = -0b1.00*2^2 = -4.0
0x05 0_00001_01 = +0b1.01*2^-14 = ~7.63e-05,0x45 0_10001_01 = +0b1.01*2^2 = 5.0,0x85 1_00001_01 = -0b1.01*2^-14 = ~-7.63e-05,0xc5 1_10001_01 = -0b1.01*2^2 = -5.0
0x06 0_00001_10 = +0b1.10*2^-14 = ~9.16e-05,0x46 0_10001_10 = +0b1.10*2^2 = 6.0,0x86 1_00001_10 = -0b1.10*2^-14 = ~-9.16e-05,0xc6 1_10001_10 = -0b1.10*2^2 = -6.0
0x07 0_00001_11 = +0b1.11*2^-14 = ~0.000,0x47 0_10001_11 = +0b1.11*2^2 = 7.0,0x87 1_00001_11 = -0b1.11*2^-14 = ~-0.000,0xc7 1_10001_11 = -0b1.11*2^2 = -7.0
0x08 0_00010_00 = +0b1.00*2^-13 = ~0.000,0x48 0_10010_00 = +0b1.00*2^3 = 8.0,0x88 1_00010_00 = -0b1.00*2^-13 = ~-0.000,0xc8 1_10010_00 = -0b1.00*2^3 = -8.0
0x09 0_00010_01 = +0b1.01*2^-13 = ~0.000,0x49 0_10010_01 = +0b1.01*2^3 = 10.0,0x89 1_00010_01 = -0b1.01*2^-13 = ~-0.000,0xc9 1_10010_01 = -0b1.01*2^3 = -10.0
0x0a 0_00010_10 = +0b1.10*2^-13 = ~0.000,0x4a 0_10010_10 = +0b1.10*2^3 = 12.0,0x8a 1_00010_10 = -0b1.10*2^-13 = ~-0.000,0xca 1_10010_10 = -0b1.10*2^3 = -12.0


In [8]:
# What would the table look like without subnormals
fi = format_info_p3109(4)
fi.name += ' [without subnormals]'
fi.has_subnormals = False
HTML(mktbl(fi, short=True, vs_width=8, vs_d=3))

0x00 0_0000_000 = 0.0,0x40 0_1000_000 = +0b1.000*2^0 = 1.0,0x80 1_0000_000 = nan,0xc0 1_1000_000 = -0b1.000*2^0 = -1.0
0x01 0_0000_001 = +0b1.001*2^-8 = ~0.004,0x41 0_1000_001 = +0b1.001*2^0 = 1.125,0x81 1_0000_001 = -0b1.001*2^-8 = ~-0.004,0xc1 1_1000_001 = -0b1.001*2^0 = -1.125
0x02 0_0000_010 = +0b1.010*2^-8 = ~0.005,0x42 0_1000_010 = +0b1.010*2^0 = 1.25,0x82 1_0000_010 = -0b1.010*2^-8 = ~-0.005,0xc2 1_1000_010 = -0b1.010*2^0 = -1.25
0x03 0_0000_011 = +0b1.011*2^-8 = ~0.005,0x43 0_1000_011 = +0b1.011*2^0 = 1.375,0x83 1_0000_011 = -0b1.011*2^-8 = ~-0.005,0xc3 1_1000_011 = -0b1.011*2^0 = -1.375
0x04 0_0000_100 = +0b1.100*2^-8 = ~0.006,0x44 0_1000_100 = +0b1.100*2^0 = 1.5,0x84 1_0000_100 = -0b1.100*2^-8 = ~-0.006,0xc4 1_1000_100 = -0b1.100*2^0 = -1.5
0x05 0_0000_101 = +0b1.101*2^-8 = ~0.006,0x45 0_1000_101 = +0b1.101*2^0 = 1.625,0x85 1_0000_101 = -0b1.101*2^-8 = ~-0.006,0xc5 1_1000_101 = -0b1.101*2^0 = -1.625
0x06 0_0000_110 = +0b1.110*2^-8 = ~0.007,0x46 0_1000_110 = +0b1.110*2^0 = 1.75,0x86 1_0000_110 = -0b1.110*2^-8 = ~-0.007,0xc6 1_1000_110 = -0b1.110*2^0 = -1.75
0x07 0_0000_111 = +0b1.111*2^-8 = ~0.007,0x47 0_1000_111 = +0b1.111*2^0 = 1.875,0x87 1_0000_111 = -0b1.111*2^-8 = ~-0.007,0xc7 1_1000_111 = -0b1.111*2^0 = -1.875
0x08 0_0001_000 = +0b1.000*2^-7 = ~0.008,0x48 0_1001_000 = +0b1.000*2^1 = 2.0,0x88 1_0001_000 = -0b1.000*2^-7 = ~-0.008,0xc8 1_1001_000 = -0b1.000*2^1 = -2.0
0x09 0_0001_001 = +0b1.001*2^-7 = ~0.009,0x49 0_1001_001 = +0b1.001*2^1 = 2.25,0x89 1_0001_001 = -0b1.001*2^-7 = ~-0.009,0xc9 1_1001_001 = -0b1.001*2^1 = -2.25
0x0a 0_0001_010 = +0b1.010*2^-7 = ~0.010,0x4a 0_1001_010 = +0b1.010*2^1 = 2.5,0x8a 1_0001_010 = -0b1.010*2^-7 = ~-0.010,0xca 1_1001_010 = -0b1.010*2^1 = -2.5


In [9]:
## Write HTML to disk
import pathlib
dir = pathlib.Path("html")
dir.mkdir(parents=True, exist_ok=True)

a = airium.Airium()
a('<!DOCTYPE html>')
a('<!-- Autogenerated from make-value-tables.ipynb -->')
a.head().title(_t='F8 value tables')
with a.body():
  a.h2(_t='F8 value tables')
  with a.ol():
    for fi in p3109_and_ocp_formats:
      html_str = mktbl(fi)
      filename = f"value-table-{fi.name}.html"
      print(f'Saving to {dir / filename}')
      a.li().a(href=filename).code(_t=fi.name)
      with open(dir / filename, "w") as f:
        f.write(html_str)

index_filename = dir / 'index.html'
print(f'Saving {index_filename}')
with open(index_filename, "w") as index:
  index.write(str(a))


Saving to html/value-table-p3109_p1.html


Saving to html/value-table-p3109_p2.html
Saving to html/value-table-p3109_p3.html
Saving to html/value-table-p3109_p4.html
Saving to html/value-table-p3109_p5.html
Saving to html/value-table-p3109_p6.html
Saving to html/value-table-p3109_p7.html
Saving to html/value-table-ocp_e5m2.html
Saving to html/value-table-ocp_e4m3.html
Saving html/index.html


## Show full table for p=4

In [10]:
HTML(mktbl(format_info_p3109(4), short=True))

0x00 0_0000_000 = 0.0,0x40 0_1000_000 = +0b1.000*2^0 = 1.0,0x80 1_0000_000 = nan,0xc0 1_1000_000 = -0b1.000*2^0 = -1.0
0x01 0_0000_001 = +0b0.001*2^-7 = 0.0009765625,0x41 0_1000_001 = +0b1.001*2^0 = 1.125,0x81 1_0000_001 = -0b0.001*2^-7 = -0.0009765625,0xc1 1_1000_001 = -0b1.001*2^0 = -1.125
0x02 0_0000_010 = +0b0.010*2^-7 = 0.001953125,0x42 0_1000_010 = +0b1.010*2^0 = 1.25,0x82 1_0000_010 = -0b0.010*2^-7 = -0.001953125,0xc2 1_1000_010 = -0b1.010*2^0 = -1.25
0x03 0_0000_011 = +0b0.011*2^-7 = 0.0029296875,0x43 0_1000_011 = +0b1.011*2^0 = 1.375,0x83 1_0000_011 = -0b0.011*2^-7 = -0.0029296875,0xc3 1_1000_011 = -0b1.011*2^0 = -1.375
0x04 0_0000_100 = +0b0.100*2^-7 = 0.00390625,0x44 0_1000_100 = +0b1.100*2^0 = 1.5,0x84 1_0000_100 = -0b0.100*2^-7 = -0.00390625,0xc4 1_1000_100 = -0b1.100*2^0 = -1.5
0x05 0_0000_101 = +0b0.101*2^-7 = 0.0048828125,0x45 0_1000_101 = +0b1.101*2^0 = 1.625,0x85 1_0000_101 = -0b0.101*2^-7 = -0.0048828125,0xc5 1_1000_101 = -0b1.101*2^0 = -1.625
0x06 0_0000_110 = +0b0.110*2^-7 = 0.005859375,0x46 0_1000_110 = +0b1.110*2^0 = 1.75,0x86 1_0000_110 = -0b0.110*2^-7 = -0.005859375,0xc6 1_1000_110 = -0b1.110*2^0 = -1.75
0x07 0_0000_111 = +0b0.111*2^-7 = 0.0068359375,0x47 0_1000_111 = +0b1.111*2^0 = 1.875,0x87 1_0000_111 = -0b0.111*2^-7 = -0.0068359375,0xc7 1_1000_111 = -0b1.111*2^0 = -1.875
0x08 0_0001_000 = +0b1.000*2^-7 = 0.0078125,0x48 0_1001_000 = +0b1.000*2^1 = 2.0,0x88 1_0001_000 = -0b1.000*2^-7 = -0.0078125,0xc8 1_1001_000 = -0b1.000*2^1 = -2.0
0x09 0_0001_001 = +0b1.001*2^-7 = 0.0087890625,0x49 0_1001_001 = +0b1.001*2^1 = 2.25,0x89 1_0001_001 = -0b1.001*2^-7 = -0.0087890625,0xc9 1_1001_001 = -0b1.001*2^1 = -2.25
0x0a 0_0001_010 = +0b1.010*2^-7 = 0.009765625,0x4a 0_1001_010 = +0b1.010*2^1 = 2.5,0x8a 1_0001_010 = -0b1.010*2^-7 = -0.009765625,0xca 1_1001_010 = -0b1.010*2^1 = -2.5


## Format tables: OCP formats

In [11]:
HTML(mktbl(format_info_ocp_e5m2, short=True))

0x00 0_00000_00 = 0.0,0x40 0_10000_00 = +0b1.00*2^1 = 2.0,0x80 1_00000_00 = -0.0,0xc0 1_10000_00 = -0b1.00*2^1 = -2.0
0x01 0_00000_01 = +0b0.01*2^-14 = ~1.5258789e-05,0x41 0_10000_01 = +0b1.01*2^1 = 2.5,0x81 1_00000_01 = -0b0.01*2^-14 = ~-1.5258789e-05,0xc1 1_10000_01 = -0b1.01*2^1 = -2.5
0x02 0_00000_10 = +0b0.10*2^-14 = ~3.0517578e-05,0x42 0_10000_10 = +0b1.10*2^1 = 3.0,0x82 1_00000_10 = -0b0.10*2^-14 = ~-3.0517578e-05,0xc2 1_10000_10 = -0b1.10*2^1 = -3.0
0x03 0_00000_11 = +0b0.11*2^-14 = ~4.5776367e-05,0x43 0_10000_11 = +0b1.11*2^1 = 3.5,0x83 1_00000_11 = -0b0.11*2^-14 = ~-4.5776367e-05,0xc3 1_10000_11 = -0b1.11*2^1 = -3.5
0x04 0_00001_00 = +0b1.00*2^-14 = ~6.1035156e-05,0x44 0_10001_00 = +0b1.00*2^2 = 4.0,0x84 1_00001_00 = -0b1.00*2^-14 = ~-6.1035156e-05,0xc4 1_10001_00 = -0b1.00*2^2 = -4.0
0x05 0_00001_01 = +0b1.01*2^-14 = ~7.6293945e-05,0x45 0_10001_01 = +0b1.01*2^2 = 5.0,0x85 1_00001_01 = -0b1.01*2^-14 = ~-7.6293945e-05,0xc5 1_10001_01 = -0b1.01*2^2 = -5.0
0x06 0_00001_10 = +0b1.10*2^-14 = ~9.1552734e-05,0x46 0_10001_10 = +0b1.10*2^2 = 6.0,0x86 1_00001_10 = -0b1.10*2^-14 = ~-9.1552734e-05,0xc6 1_10001_10 = -0b1.10*2^2 = -6.0
0x07 0_00001_11 = +0b1.11*2^-14 = ~0.00010681,0x47 0_10001_11 = +0b1.11*2^2 = 7.0,0x87 1_00001_11 = -0b1.11*2^-14 = ~-0.00010681,0xc7 1_10001_11 = -0b1.11*2^2 = -7.0
0x08 0_00010_00 = +0b1.00*2^-13 = ~0.00012207,0x48 0_10010_00 = +0b1.00*2^3 = 8.0,0x88 1_00010_00 = -0b1.00*2^-13 = ~-0.00012207,0xc8 1_10010_00 = -0b1.00*2^3 = -8.0
0x09 0_00010_01 = +0b1.01*2^-13 = ~0.00015259,0x49 0_10010_01 = +0b1.01*2^3 = 10.0,0x89 1_00010_01 = -0b1.01*2^-13 = ~-0.00015259,0xc9 1_10010_01 = -0b1.01*2^3 = -10.0
0x0a 0_00010_10 = +0b1.10*2^-13 = ~0.00018311,0x4a 0_10010_10 = +0b1.10*2^3 = 12.0,0x8a 1_00010_10 = -0b1.10*2^-13 = ~-0.00018311,0xca 1_10010_10 = -0b1.10*2^3 = -12.0


In [12]:
HTML(mktbl(format_info_ocp_e4m3, short=True))

0x00 0_0000_000 = 0.0,0x40 0_1000_000 = +0b1.000*2^1 = 2.0,0x80 1_0000_000 = -0.0,0xc0 1_1000_000 = -0b1.000*2^1 = -2.0
0x01 0_0000_001 = +0b0.001*2^-6 = 0.001953125,0x41 0_1000_001 = +0b1.001*2^1 = 2.25,0x81 1_0000_001 = -0b0.001*2^-6 = -0.001953125,0xc1 1_1000_001 = -0b1.001*2^1 = -2.25
0x02 0_0000_010 = +0b0.010*2^-6 = 0.00390625,0x42 0_1000_010 = +0b1.010*2^1 = 2.5,0x82 1_0000_010 = -0b0.010*2^-6 = -0.00390625,0xc2 1_1000_010 = -0b1.010*2^1 = -2.5
0x03 0_0000_011 = +0b0.011*2^-6 = 0.005859375,0x43 0_1000_011 = +0b1.011*2^1 = 2.75,0x83 1_0000_011 = -0b0.011*2^-6 = -0.005859375,0xc3 1_1000_011 = -0b1.011*2^1 = -2.75
0x04 0_0000_100 = +0b0.100*2^-6 = 0.0078125,0x44 0_1000_100 = +0b1.100*2^1 = 3.0,0x84 1_0000_100 = -0b0.100*2^-6 = -0.0078125,0xc4 1_1000_100 = -0b1.100*2^1 = -3.0
0x05 0_0000_101 = +0b0.101*2^-6 = 0.009765625,0x45 0_1000_101 = +0b1.101*2^1 = 3.25,0x85 1_0000_101 = -0b0.101*2^-6 = -0.009765625,0xc5 1_1000_101 = -0b1.101*2^1 = -3.25
0x06 0_0000_110 = +0b0.110*2^-6 = 0.01171875,0x46 0_1000_110 = +0b1.110*2^1 = 3.5,0x86 1_0000_110 = -0b0.110*2^-6 = -0.01171875,0xc6 1_1000_110 = -0b1.110*2^1 = -3.5
0x07 0_0000_111 = +0b0.111*2^-6 = 0.013671875,0x47 0_1000_111 = +0b1.111*2^1 = 3.75,0x87 1_0000_111 = -0b0.111*2^-6 = -0.013671875,0xc7 1_1000_111 = -0b1.111*2^1 = -3.75
0x08 0_0001_000 = +0b1.000*2^-6 = 0.015625,0x48 0_1001_000 = +0b1.000*2^2 = 4.0,0x88 1_0001_000 = -0b1.000*2^-6 = -0.015625,0xc8 1_1001_000 = -0b1.000*2^2 = -4.0
0x09 0_0001_001 = +0b1.001*2^-6 = 0.017578125,0x49 0_1001_001 = +0b1.001*2^2 = 4.5,0x89 1_0001_001 = -0b1.001*2^-6 = -0.017578125,0xc9 1_1001_001 = -0b1.001*2^2 = -4.5
0x0a 0_0001_010 = +0b1.010*2^-6 = 0.01953125,0x4a 0_1001_010 = +0b1.010*2^2 = 5.0,0x8a 1_0001_010 = -0b1.010*2^-6 = -0.01953125,0xca 1_1001_010 = -0b1.010*2^2 = -5.0


## Generate LaTeX table of min/max values

In [13]:
df_pow2[["minSubnormal","maxSubnormal","minNormal","maxNormal","maxFinite"]]


,minSubnormal,maxSubnormal,minNormal,maxNormal,maxFinite
0,nan,nan,1*2^-62,1*2^63,1*2^63
1,1*2^-32,1*2^-32,1*2^-31,1*2^31,1*2^31
2,1*2^-17,3/2*2^-16,1*2^-15,3/2*2^15,3/2*2^15
3,1*2^-10,7/4*2^-8,1*2^-7,7/4*2^7,7/4*2^7
4,1*2^-7,15/8*2^-4,1*2^-3,15/8*2^3,15/8*2^3
5,1*2^-6,31/16*2^-2,1*2^-1,31/16*2^1,31/16*2^1
6,1*2^-6,63/32*2^-1,1*2^0,63/32*2^0,63/32*2^0
7,1*2^-16,3/2*2^-15,1*2^-14,7/4*2^15,7/4*2^15
8,1*2^-9,7/4*2^-7,1*2^-6,7/4*2^8,7/4*2^8


In [14]:
import pathlib
dir = pathlib.Path("latex")
dir.mkdir(parents=True, exist_ok=True)

tbl3 = df_pow2[["name", "minSubnormal","maxSubnormal","minNormal","maxNormal","maxFinite"]]
header=["Format", "minSubnormal","maxSubnormal","minNormal","maxNormal","maxFinite"]
latex = tbl3.to_latex(header=header, index=False, float_format=gfloat.printing.float_pow2str, na_rep='N/A')

import re 
latex = re.sub('([\d./]+)\*2.textasciicircum ([\d-]+)','$\\\\binaryfloat{\\1}{\\2}$', latex)
latex = latex.replace('p3109_p', 'p')
latex = latex.replace('ocp_', 'ocp\_')

with open('latex/tbl-extremalvalues.tex', 'w') as f:
  pr = lambda x: print(x,file=f)
  pr('% File: tbl-extremalvalues.tex')
  pr('% Generated from https://github.com/P3109/Public/blob/main/Value%20Tables/make-value-tables.ipynb')
  pr(latex)

!grep -H . latex/tbl-extremalvalues.tex

latex/tbl-extremalvalues.tex:% File: tbl-extremalvalues.tex
latex/tbl-extremalvalues.tex:% Generated from https://github.com/P3109/Public/blob/main/Value%20Tables/make-value-tables.ipynb
latex/tbl-extremalvalues.tex:\begin{tabular}{llllll}
latex/tbl-extremalvalues.tex:\toprule
latex/tbl-extremalvalues.tex:Format & minSubnormal & maxSubnormal & minNormal & maxNormal & maxFinite \\
latex/tbl-extremalvalues.tex:\midrule
latex/tbl-extremalvalues.tex:p1 & nan & nan & 1*2^-62 & 1*2^63 & 1*2^63 \\
latex/tbl-extremalvalues.tex:p2 & 1*2^-32 & 1*2^-32 & 1*2^-31 & 1*2^31 & 1*2^31 \\
latex/tbl-extremalvalues.tex:p3 & 1*2^-17 & 3/2*2^-16 & 1*2^-15 & 3/2*2^15 & 3/2*2^15 \\
latex/tbl-extremalvalues.tex:p4 & 1*2^-10 & 7/4*2^-8 & 1*2^-7 & 7/4*2^7 & 7/4*2^7 \\
latex/tbl-extremalvalues.tex:p5 & 1*2^-7 & 15/8*2^-4 & 1*2^-3 & 15/8*2^3 & 15/8*2^3 \\
latex/tbl-extremalvalues.tex:p6 & 1*2^-6 & 31/16*2^-2 & 1*2^-1 & 31/16*2^1 & 31/16*2^1 \\
latex/tbl-extremalvalues.tex:p7 & 1*2^-6 & 63/32*2^-1 & 1*2^0 & 63/32*

## LaTeX value tables

In [15]:

def mktbltex(fi : FormatInfo, file):
  # Make tables
  cols = 4
  rows = 256//cols

  for i in range(0,rows):
    out_row = []
    for n in range(i,256,rows):
      fv = decode_float(fi, n)
      text = str_tablerow(fi, fv)
      style = table_style(fv)

      # 2^-7  -> \pow{-7}
      text = re.sub(r'\*2\^([-0-9]+)', r'\\pow{\1}', text)
      # 1.234E7  -> \e{1.234}{7}
      text = re.sub(r'([0-9.+-]+)[eE]([0-9]+)', r'\\e{\1}{\2}', text)
      # 1.234E-7  -> \e{1.234}{-7}
      text = re.sub(r'([0-9.+-]+)[eE]-([0-9]+)', r'\\e{\1}{\\neg{\2}}', text)
      # -1.234 EOL  -> \f{-1.234}
      text = re.sub(r'([0-9.+-]+)$', r'\\f{\1}', text)
      #\f{-1.234} -> \f{\neg{1.234}}
      text = re.sub(r'\\(f|pow)\{-([0-9.]+)\}', r'\\\1{\\neg{\2}}', text)
      
      # ^0x44  -> 0x44~
      text = re.sub('^(0x[0-9a-f]+) ', '\\1 = ', text)
      text = text.replace('_', '\_')
      text = text.replace('= ~', '\\approx ')
      out_row += [f'\\{style}{{{text}}}']
    print(*out_row, sep="&\n", end=r'\\'+"\n", file=file)

for fi in p3109_and_ocp_formats:
  filename = f"value-table-{fi.name}.tex"
  print(f'Saving to {dir / filename}')
  with open(dir / filename, "w") as f:
    print('% Autogenerated from make-value-tables.ipynb', file=f)
    mktbltex(fi, f)


Saving to latex/value-table-p3109_p1.tex
Saving to latex/value-table-p3109_p2.tex
Saving to latex/value-table-p3109_p3.tex
Saving to latex/value-table-p3109_p4.tex
Saving to latex/value-table-p3109_p5.tex
Saving to latex/value-table-p3109_p6.tex
Saving to latex/value-table-p3109_p7.tex
Saving to latex/value-table-ocp_e5m2.tex
Saving to latex/value-table-ocp_e4m3.tex
